In [21]:
import json
import numpy as np
import pandas as pd
import regex
import os

In [22]:
def get_tracing(path):
    with open(path, 'r') as f:
        json_data = json.load(f)
    columns=["Thread","Time","Event"]
    result_df=pd.DataFrame(columns=columns)
    def _get_df_row(term):
        df_row={}
        if term["name"] == "Initialize" and term["pid"] == "All":
            df_row["Thread"]= term["pid"]
            df_row["Time"]= term["ts"]
            df_row["Event"]= "Start"
        elif term["name"] == "Finalize" and term["pid"] == "All":
            df_row["Thread"]= term["pid"]
            df_row["Time"]= term["ts"]
            df_row["Event"]= "End"
        elif term["tid"] == "Parallel Region" and term["pid"] == "All":
            df_row["Thread"]= term["pid"]
            df_row["Time"]= term["ts"]
            df_row["Event"]= term["tid"]
        else:
            thread_id = int(term["pid"].split(" ")[1])
            df_row["Thread"]= thread_id
            df_row["Time"]= term["ts"]
            df_row["Event"]= term["tid"]
        return df_row
    for term,i in zip(json_data,range(len(json_data))):
        result_df.loc[i]=_get_df_row(term)
    return result_df

In [24]:
# The loggest parallel thread time / sequential thread time
def thread_ratio(df):
    df=df[(df["Event"]=="Thread Region")]
    df = df.sort_values(by=["Thread","Time"])
    df = df.reset_index(drop=True)
    num_threads=np.sort(np.int64(df["Thread"].unique()))
    def _thread_time(df,i):
        tmp_df=df[df["Thread"]==i]
        tmp_df.reset_index(drop=True)
        tmp_time=0
        assert len(tmp_df)%2==0
        for j in range(0,len(tmp_df),2):
            tmp_time+=tmp_df.iloc[j+1]["Time"]-tmp_df.iloc[j]["Time"]
        return tmp_time
    seq_time=_thread_time(df,0)
    worker_time=[]
    for i in num_threads[1:]:
        worker_time.append(_thread_time(df,i))
    return max(worker_time)/seq_time

In [25]:
def overhead_ratio(df):
    df=result
    df_par=df[df["Event"]=="Parallel Region"]
    df_par=df_par.sort_values(by=["Thread","Time"])
    df_par=df_par.reset_index(drop=True)

    df_work=df[df["Event"]=="Work Region"]
    df_work=df_work.sort_values(by=["Thread","Time"])
    df_work=df_work.reset_index(drop=True)    

    df_total=df[(df["Event"]=="Thread Region")]
    num_threads=np.sort(np.int64(df_total["Thread"].unique()))
    def _diff(i):
        tmp=0
        tmp_df=df_work[df_work["Thread"]==i]
        for j in range(0,len(df_par),2):
            tmp+=tmp_df.iloc[j]["Time"]-df_par.iloc[j]["Time"]
        return tmp
    par_time=0
    for j in range(0,len(df_par.index),2):
        par_time+=df_par.iloc[j+1]["Time"]-df_par.iloc[j]["Time"]
    overhead_time=[]
    for i in num_threads:
        overhead_time.append(_diff(i))
    return max(overhead_time)/par_time
    

In [26]:
def sync_ratio(df):
    df_par=df[df["Event"]=="Parallel Region"]
    df_par=df_par.sort_values(by=["Thread","Time"])
    df_par=df_par.reset_index(drop=True)
    
    df_work=df[df["Event"]=="Work Region"]
    df_work=df_work.sort_values(by=["Thread","Time"])
    df_work=df_work.reset_index(drop=True)    

    df_total=df[(df["Event"]=="Thread Region")]
    num_threads=np.sort(np.int64(df_total["Thread"].unique()))
    def _diff(i):
        tmp=0
        tmp_df=df_work[df_work["Thread"]==i]
        for j in range(1,len(df_par.index),2):
            tmp+=df_par.iloc[j]["Time"]-tmp_df.iloc[j]["Time"]
        return tmp
    par_time=0
    for j in range(0,len(df_par.index),2):
        par_time+=df_par.iloc[j+1]["Time"]-df_par.iloc[j]["Time"]
    sync_time=[]
    for i in num_threads:
        sync_time.append(_diff(i))
    return max(sync_time)/par_time

In [27]:
def parallel_region_ratio(df):
    run_time=df[df["Event"]=="End"]["Time"].values[0]-df[df["Event"]=="Start"]["Time"].values[0]
    tmp_df=df[(df["Thread"]=="All") & (df["Event"]=="Parallel Region")]
    tmp_df=tmp_df.sort_values(by=["Time"])
    assert(tmp_df.shape[0]%2==0)
    parallel_time=0
    for i in range(0,tmp_df.shape[0],2):
        start_time=tmp_df.iloc[i]["Time"]
        end_time=tmp_df.iloc[i+1]["Time"]
        parallel_time+=end_time-start_time
    return parallel_time/run_time

In [31]:
result_balance=get_tracing("./test_balance/omptracing_24.json")
result_dynamic=get_tracing("./test_balance_dynamic/omptracing_43.json")
result_imbalance=get_tracing("./test_imbalance/omptracing_63.json")

In [32]:
# parallel_region_ratio: parallel region time / total time, for scaling check
# overhead_ratio: overhead/parallel region time, for sync check
# sync_ratio: symc/parallel region, for sync check
# sync_ratio+overhead_ratio, for work region check
# Order: balance, dynamic, imbalance

parallel_data=[parallel_region_ratio(result_balance),parallel_region_ratio(result_dynamic),parallel_region_ratio(result_imbalance)]
overhead_data=[overhead_ratio(result_balance),overhead_ratio(result_dynamic),overhead_ratio(result_imbalance)]
sync_data=[sync_ratio(result_balance),sync_ratio(result_dynamic),sync_ratio(result_imbalance)]

In [ ]:
#parallel_region_ratio = [0.5595858576527958, 0.987454394371793, 0.3173771815071966]
#overhead_ratio =[7.934512458200731e-05, 7.934512458200731e-05, 7.934512458200731e-05]
#sync_ratio =[0.15670215870589418, 3.0044310647713796e-05, 0.10504855823462356]

In [30]:
def print_res(df):
    print(thread_ratio(df))
    print(overhead_ratio(df))
    print(sync_ratio(df))
    print(parallel_region_ratio(df))
print_res()

0.9875702203875961
7.934512458200731e-05
3.0044310647713796e-05
0.987454394371793


In [22]:
def get_valgrind(path):
    # path="/home/yiwei/Bottleneck-Predictions-of-Parallel-Programs/valgrind_sample.txt"
    logfile=open(path,"r")
    line_list=logfile.readlines()
    line_len=len(line_list)
    columns=["Ir","I1mr","ILmr","Dr","D1mr","DLmr","Dw","D1mw","DLmw","Function"]
    df_counts=pd.DataFrame(columns=columns,dtype='object')
    df_ratios=pd.DataFrame(columns=columns,dtype='object')
    # The stats of all data
    head_index=0
    for line_index in range(line_len):
        m=regex.match(r"Ir                 I1mr           ILmr           Dr               D1mr           DLmr           Dw               D1mw           DLmw",line_list[line_index])
        if m!=None:
            head_index=line_index
            break
    assert head_index!=0
    
    def _read_valgrind_line(line):
        tmp_list=regex.split(r"\(|\)| ",line)
        tmp_list = list(filter(None, tmp_list))
        if len(tmp_list)<10:
            return None,None,None
        counts=[]
        ratios=[]
        k=0
        for i in range(9):
            counts.append(int(tmp_list[k].replace(",","")))
            k=k+1
            tmp_ratio=tmp_list[k]
            if tmp_ratio[-1]=="%":
                ratios.append(float(tmp_ratio[:-1])/100)
                k=k+1
            else:
                ratios.append(0.0)
        if len(counts)==0:
            return None,None,None
        func=tmp_list[-1].replace("\n","")
        if len(func)>0 and func[0]==".":
            func=None
        return counts,ratios,func
    
    counts,ratios,func=_read_valgrind_line(line_list[head_index+2])
    df_counts.loc[0]=pd.Series(counts,index=columns[:-1],dtype='object')
    df_counts.loc[0]["Function"]=func
    df_ratios.loc[0]=pd.Series(ratios,index=columns[:-1],dtype='object')
    df_ratios.loc[0]["Function"]=func

    # The stats of functions
    begin_index=head_index+2
    for line_index in range(head_index+2,line_len):
        m=regex.match(r"Ir                 I1mr         ILmr         Dr               D1mr         DLmr         Dw              D1mw           DLmw          file:function",line_list[line_index])
        if m!=None:
            begin_index=line_index
            break
    assert begin_index!=0


    for line_index in range(begin_index+3,line_len):
    # for line_index in [23]:
        counts,ratios,func=_read_valgrind_line(line_list[line_index])
        if counts==None:
            end_index=line_index
            break
        else:
            k=len(df_counts)
            df_counts.loc[k]=pd.Series(counts,index=columns[:-1],dtype='object')
            df_counts.loc[k]["Function"]=func
            df_ratios.loc[k]=pd.Series(ratios,index=columns[:-1],dtype='object')
            df_ratios.loc[k]["Function"]=func
    return df_counts,df_ratios


In [7]:
test_case = "./test_imbalance/"

In [8]:
def get_ratio_sync_paral(test_case):
    """ input test_case: the path of the test case"""
    jsonfiles = [file for file in os.listdir(test_case) if file.endswith(".json")]
    ratios = []
    for jsonfile in jsonfiles:
        print(jsonfile)
        result = get_tracing(os.path.join(test_case, jsonfile))
        parallel_ratio = parallel_region_ratio(result)
        s_ratio = sync_ratio(result)
        print("parllel region ratio:", parallel_ratio)
        print("sync region ratio   :", s_ratio)
        print("thread ratio        :", thread_ratio(result))
        print("sync / parallel:", s_ratio/parallel_ratio)
        ratios.append(s_ratio/parallel_ratio)
    print(ratios)
    avg_ratio = np.array(ratios).mean()
    return avg_ratio

In [9]:
get_ratio_sync_paral(test_case="./test_imbalance/")

[]


/tmp/ipykernel_11079/474318403.py:16: RuntimeWarning: Mean of empty slice.
  avg_ratio = np.array(ratios).mean()
/home/yiwei/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

In [10]:
get_ratio_sync_paral(test_case="./test_balance/")

[]


/tmp/ipykernel_11079/474318403.py:16: RuntimeWarning: Mean of empty slice.
  avg_ratio = np.array(ratios).mean()


nan

In [11]:
get_ratio_sync_paral(test_case="./test_imbalance_nowait/")

omptracing_30.json


ValueError: max() arg is an empty sequence

In [ ]:
get_ratio_sync_paral(test_case="./test_balance_dynamic/") # for showcase thread overhead bottleneck

: 

In [ ]:
get_ratio_sync_paral(test_case="./test_synchronization/") # omp scan

: 

In [ ]:
df_counts,df_ratios=get_valgrind("./valgrind_sample.txt")

: 